In [1]:
# 0. 사용할 패키지 불러오기
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input, Embedding
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
import csv
import random

# 랜덤시드 고정시키기
np.random.seed(5)
   
MAX_IDX = 275
MAX_GRID = 275
FEATURE_NUM = 15
TOTAL_DATA = MAX_GRID * MAX_IDX
WINDOW_SIZE = 4  #TODO :: edit

# 손실 이력 클래스 정의
class LossHistory(keras.callbacks.Callback):
    def init(self):
        self.losses = []
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

# 그리드에서 랜덤으로 값 받아오는 함수
def randomPickInGrid(Wdata, Max, GridNum):
    randidx = random.randrange(0,Max * 0.8)
    return Wdata[int((GridNum-1)*Max*0.8) + randidx]

# 경로 입력하면 데이터 만들어서 넣어주는 함수
def genData(Wdata, path):
    path_len = len(path)
    real_val = []
    for i in range(path_len):
        real_val.append(randomPickInGrid(Wdata,MAX_IDX,path[i]))
#         real_val[i][FEATURE_NUM] = int(real_val[i][FEATURE_NUM]) - 1 # 경로가 1부터 있어서 one-hot-encoding 하면 +1됨 
                                                                     # 따라서 -1 붙여준다
    return real_val

# WINDOW_SIZE에 맞게 데이터 잘라주는 함수
def makeWindowMat(seqData):
    dataset_x = []
    dataset_y = []
    for i in range(len(seqData) - WINDOW_SIZE):
        subset = seqData[i : ( i + WINDOW_SIZE + 1)]
        for si in range(len(subset) - 1):
            dataset_x.append(subset[si][:FEATURE_NUM])
        dataset_y.append(subset[WINDOW_SIZE][FEATURE_NUM:])
    return np.array(dataset_x) , np.array(dataset_y)

class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.losses = []
        self.vol_losses = []
        self.accs = []
        self.vol_accs = []        
        
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.vol_losses.append(logs.get('vol_loss'))
        self.accs.append(logs.get('acc'))
        self.vol_accs.append(logs.get('acc_loss'))

# WiFi 데이터 불러오기
f = open('./data/made_wifi4.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)

WiFiData = []
for line in rdr:
    tmp = []
    for i in range(FEATURE_NUM):
        tmp.append((line[i]))
    tmp.append(int(line[FEATURE_NUM]))
    WiFiData.append(tmp)
f.close()

# 4. 모델 학습과정 설정하기
early_stop = EarlyStopping(monitor='loss', patience=5) # 조기종료

from keras.models import load_model
model = load_model('wifi_LSTM_model4.h5')
temp_weights = [layer.get_weights() for layer in model.layers]

# print(temp_weights)

# Define the model
model = keras.models.Sequential()


model.add(LSTM(128, batch_input_shape = (1, 4, 15), stateful=True, name = "input_node"))
# model.add(LSTM(128, input_shape = (WINDOW_SIZE, FEATURE_NUM), stateful=False)) # stateless 테스트용
model.add(Dense(275 + 1, activation='softmax', name = "output_node"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

for i in range(len(temp_weights)):
    model.layers[i].set_weights(temp_weights[i])
    
# 1. 데이터 준비하기
f = open('path4.csv', 'r', encoding='utf-8-sig')
rdr = csv.reader(f)

path_idx = 0
path_choice = [1] ## 학습 할 path만 설정

for line in rdr:
    tmp = []
    path_idx += 1
    if path_idx not in path_choice: ## 중간에 몇개의 path만 실행한다
        continue
    for i in range(len(line)):
        tmp.append(int(line[i]))
    path = tmp
    print("path's length : " + str(len(path)) + ", start : " + str(path[0]) + ", end : " + str(path[len(path)-1]))

    # 5. 모델 학습시키기
    num_epochs = 1

    history = LossHistory() # 손실 이력 객체 생성
    history.init()

    model.summary()

    custom_hist = CustomHistory()
    custom_hist.init()
    
    for epoch_idx in range(num_epochs):
        trainData = genData(WiFiData,path)
        x_train, y_train = makeWindowMat(trainData)
        x_train = np.reshape(x_train, (len(path) - WINDOW_SIZE, WINDOW_SIZE, FEATURE_NUM))
        y_train = np_utils.to_categorical(y_train, num_classes=276)
        one_hot_vec_size = y_train.shape[1]
        print (str(path_idx) + "'s path epochs : " + str(epoch_idx) )
        tb_hist = keras.callbacks.TensorBoard(log_dir='./graph2', histogram_freq=0, write_graph=True, write_images=True)
        model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2, shuffle=False, callbacks=[tb_hist]) # 50 is X.shape[0]
        model.reset_states()

    # stateless Test한거임
    # model.fit(x_train,y_train, epochs= 2000, batch_size=4, verbose = 1)

    # 6. 학습과정 살펴보기
#     %matplotlib inline
#     import matplotlib.pyplot as plt

#     plt.plot(history.losses)
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train'], loc='upper left')
#     plt.show()

    # 7. 모델 평가하기
    scores = model.evaluate(x_train, y_train, batch_size=1)
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    model.reset_states()
    
f.close()

# model.save('wifi_LSTM_model4.h5')
## TODO:: 저장된 이후부터 모든 경로 학습하는 코드

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
path's length : 80, start : 141, end : 142
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_node (LSTM)            (1, 128)                  73728     
_________________________________________________________________
output_node (Dense)          (1, 276)                  35604     
Total params: 109,332
Trainable params: 109,332
Non-trainable params: 0
_________________________________________________________________
1's path epochs : 0
Epoch 1/1
0s - loss: 15.4484 - acc: 0.0395
64/76 [========================>.....] - ETA: 0sacc: 3.95%
